In [13]:
import pandas as pd
from sklearn.cluster import KMeans

data = pd.read_excel('session.xlsx', parse_dates=['SESSION_END', 'SESSION_START'])

# Remove sessions with a duration of 0 seconds
data = data[data['SESSION_DURATION'] > 0]

# Remove outliers using the IQR method
Q1 = data['SESSION_DURATION'].quantile(0.25)
Q3 = data['SESSION_DURATION'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

data_filtered = data[(data['SESSION_DURATION'] >= lower_bound) & (data['SESSION_DURATION'] <= upper_bound)]

# Perform clustering analysis
X = data_filtered['SESSION_DURATION'].values.reshape(-1, 1)
kmeans = KMeans(n_clusters=4, random_state=42).fit(X)

# Add cluster labels to the DataFrame
data_filtered['Cluster'] = kmeans.labels_

df = data_filtered

/var/folders/6z/bh7_dtkj0c7dqhfn_nwgx23r0000gn/T/ipykernel_70218/2967216226.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_filtered['Cluster'] = kmeans.labels_


In [12]:
import pandas as pd

# Assuming you have the original dataset loaded into a DataFrame called 'data'
# and the clustered data loaded into a DataFrame called 'df'

# Merge the cluster information with the original dataset
merged_data = pd.merge(data, df[['ID', 'SESSION', 'Cluster', 'SESSION_DURATION']], on=['ID', 'SESSION'])

# Sort merged data by session and timestamp
merged_data = merged_data.sort_values(by=['SESSION', 'TIMESTAMP'])

# Create a navigation path for each session
merged_data['NAVIGATION_PATH'] = merged_data.groupby('SESSION')['CURRENT'].transform(lambda x: ' -> '.join(x))

# Limit the navigation path to a maximum depth of 10 different screens
merged_data['NAVIGATION_PATH'] = merged_data['NAVIGATION_PATH'].apply(lambda x: ' -> '.join(pd.unique(x.split(' -> '))[:4]))

# Analyze user behavior within each cluster
user_behavior = merged_data.groupby(['ID', 'Cluster']).agg({
    'SESSION': 'count',
    'SESSION_DURATION': 'mean',
    'PART': lambda x: x.mode().iloc[0] if not x.empty else None,
    'NAVIGATION_PATH': lambda x: x.mode().iloc[0] if not x.empty else None
}).reset_index()

user_behavior.columns = ['ID', 'Cluster', 'Total Sessions', 'Avg Session Duration', 'Most Frequent Part', 'Most Common Navigation Path']

# Identify user profiles or personas
user_profiles = user_behavior.groupby('Cluster').agg({
    'Total Sessions': 'mean',
    'Avg Session Duration': 'mean',
    'Most Frequent Part': lambda x: x.mode().iloc[0] if not x.empty else None,
    'Most Common Navigation Path': lambda x: x.mode().iloc[0] if not x.empty else None
}).reset_index()

# Print the user profiles
print("User Profiles/Personas:")
print(user_profiles)

/var/folders/6z/bh7_dtkj0c7dqhfn_nwgx23r0000gn/T/ipykernel_70218/4225386360.py:16: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  merged_data['NAVIGATION_PATH'] = merged_data['NAVIGATION_PATH'].apply(lambda x: ' -> '.join(pd.unique(x.split(' -> '))[:4]))


User Profiles/Personas:
   Cluster  Total Sessions  Avg Session Duration Most Frequent Part  \
0        0       39.531250             72.356936              Tools   
1        1       53.924138             17.043269              Tools   
2        2       42.039735            268.396582              Tools   
3        3       40.272277            160.835631              Tools   

                                                                                    Most Common Navigation Path  
0                     Mainpage -> List of active calendar tasks -> List of tools -> Tools: meal rhythm: choices  
1                     Mainpage -> List of active calendar tasks -> List of tools -> Tools: meal rhythm: choices  
2  Mainpage -> List of active calendar tasks -> List of tools -> Personal_resources_questionnaire: introduction  
3            Mainpage -> List of tools -> Menu: health domain modules -> Stress_management: module introduction  
